In [1]:

# File Recognition File Encryptor

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import cv2
import numpy as np
import tkinter as tk
from tkinter import simpledialog, filedialog, messagebox
from PIL import Image, ImageTk
import face_recognition
from cryptography.fernet import Fernet
import pickle
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

USERS_FILE = "users_face.pkl"
KEYS_DIR = "keys_face"
if not os.path.exists(KEYS_DIR):
    os.mkdir(KEYS_DIR)

def load_users():
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE, "rb") as f:
            return pickle.load(f)
    return {}

def save_users(users):
    with open(USERS_FILE, "wb") as f:
        pickle.dump(users, f)

class FaceRegistrationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Register New User (Face + Username)")
        self.root.geometry("800x600")
        self.cap = None
        self.is_capturing = False

        self.label = tk.Label(root, text="Register Face\nPress 'Capture Face' when ready", font=("Helvetica", 16))
        self.label.pack(pady=10)
        self.video_label = tk.Label(root)
        self.video_label.pack()
        self.capture_btn = tk.Button(root, text="Capture Face", command=self.capture_face)
        self.capture_btn.pack(pady=10)
        self.status = tk.Label(root, text="Status: Waiting...", font=("Helvetica", 10))
        self.status.pack(pady=5)

        self.root.after(0, self.start_camera)

    def start_camera(self):
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Cannot access camera")
            return
        self.is_capturing = True
        self.update_video()

    def update_video(self):
        if not (self.is_capturing and self.cap):
            return
        ret, frame = self.cap.read()
        if not ret or frame is None:
            self.root.after(10, self.update_video)
            return

        # Display frame in GUI
        rgb_disp = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(rgb_disp)
        imgtk = ImageTk.PhotoImage(image=img)
        self.video_label.config(image=imgtk)
        self.video_label.imgtk = imgtk

        self.frame = frame  # Save for capture
        self.root.after(10, self.update_video)

    def capture_face(self):
        frame = getattr(self, 'frame', None)
        if frame is None:
            self.status.config(text="No frame to capture!")
            return

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = face_recognition.face_locations(rgb_frame)
        if not faces:
            self.status.config(text="No face detected. Try again!")
            return
        face_encodings = face_recognition.face_encodings(rgb_frame, faces)
        if not face_encodings:
            self.status.config(text="No face encoding found. Try again!")
            return
        encoding = face_encodings[0]

        username = simpledialog.askstring("Username", "Enter a unique username for this user:")
        if not username:
            self.status.config(text="Username required!")
            return
        users = load_users()
        if username in users:
            self.status.config(text="Username already exists!")
            return
        users[username] = encoding
        save_users(users)
        self.status.config(text=f"User {username} registered successfully!")
        self.is_capturing = False
        self.cap.release()
        self.root.destroy()

def encrypt_for_user():
    users = load_users()
    if not users:
        messagebox.showerror("No Users", "No users registered. Please register a user first.")
        return
    root = tk.Tk(); root.withdraw()
    username = simpledialog.askstring("Recipient Username", "Enter the username of the recipient:")
    if not username or username not in users:
        messagebox.showerror("Error", "User not found! Register user first.")
        return
    file_path = filedialog.askopenfilename(title="Select file to encrypt")
    if not file_path:
        return
    key = Fernet.generate_key()
    cipher = Fernet(key)
    with open(file_path, "rb") as f: data = f.read()
    encrypted = cipher.encrypt(data)
    enc_file = f"{username}_{os.path.basename(file_path)}.enc"
    with open(enc_file, "wb") as f: f.write(encrypted)

    # Ensure keys directory exists
    if not os.path.exists(KEYS_DIR):
        os.makedirs(KEYS_DIR)
    with open(os.path.join(KEYS_DIR, f"{username}.key"), "wb") as f: f.write(key)

    messagebox.showinfo("Success", f"File Encrypted for {username}!\nSaved: {enc_file}\nKey saved: {KEYS_DIR}/{username}.key")

def authenticate_and_decrypt():
    users = load_users()
    if not users:
        print("No users registered."); return

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot access camera"); return

    detected_username = None

    # Force authentication window on top
    cv2.namedWindow("Authentication", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Authentication", cv2.WND_PROP_TOPMOST, 1)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = face_recognition.face_locations(rgb_frame)
        if faces:
            encoding = face_recognition.face_encodings(rgb_frame, faces)[0]
            known_encodings = list(users.values())
            known_usernames = list(users.keys())
            matches = face_recognition.compare_faces(known_encodings, encoding, tolerance=0.48)
            face_distances = face_recognition.face_distance(known_encodings, encoding)
            if True in matches:
                best_match_index = np.argmin(face_distances)
                detected_username = known_usernames[best_match_index]
            # Show feedback
            cv2.rectangle(frame, (faces[0][3], faces[0][0]), (faces[0][1], faces[0][2]), (0,255,0), 2)
            info = f"User: {detected_username if detected_username else 'Not recognized'}"
            cv2.putText(frame, info, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        cv2.imshow("Authentication", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if detected_username:
            break

    cap.release()
    cv2.destroyAllWindows()

    if detected_username:
        key_path = os.path.join(KEYS_DIR, f"{detected_username}.key")
        if not os.path.exists(key_path): print("Key not found!"); return
        root = tk.Tk(); root.withdraw()
        enc_file = filedialog.askopenfilename(title="Select encrypted file", filetypes=[("Encrypted", ".enc")])
        if not enc_file:
            messagebox.showerror("Error", "No file selected.")
            return

        detected_username_clean = detected_username.strip().lower()
        enc_file_name = os.path.basename(enc_file).strip().lower()

        print(f"Detected username: '{detected_username_clean}'")
        print(f"Selected file: '{enc_file_name}'")

        if not enc_file_name.startswith(f"{detected_username_clean}_"):
            messagebox.showerror("Error", "This file is not for you or invalid selection.")
            return

        if not os.path.exists(enc_file):
            messagebox.showerror("Error", f"File not found: {enc_file}\nCheck the filename and try again.")
            return

        try:
            with open(key_path, "rb") as f:
                key = f.read()
            cipher = Fernet(key)
            with open(enc_file, "rb") as f:
                encrypted = f.read()
            decrypted = cipher.decrypt(encrypted)
            out_file = enc_file.replace(".enc", "_decrypted")
            with open(out_file, "wb") as f:
                f.write(decrypted)
            messagebox.showinfo("Success", f"File decrypted!\nSaved: {out_file}")
            root.destroy()
        except Exception as e:
            messagebox.showerror("Error", f"Decryption failed: {e}")

    else:
        messagebox.showerror("Access Denied", "Authentication Failed! Face not recognized.")

if __name__ == "__main__":
    while True:
        print("\n1. Register New User (Face + Username)")
        print("2. Encrypt File for Registered User")
        print("3. Authenticate & Decrypt File")
        print("4. Exit")
        choice = input("Enter choice (1/2/3/4): ")
        if choice == "1":
            root = tk.Tk()
            app = FaceRegistrationApp(root)
            root.mainloop()
        elif choice == "2":
            encrypt_for_user()
        elif choice == "3":
            authenticate_and_decrypt()
        elif choice == "4":
            break
        else:
            print("Invalid choice")




1. Register New User (Face + Username)
2. Encrypt File for Registered User
3. Authenticate & Decrypt File
4. Exit


Enter choice (1/2/3/4):  1



1. Register New User (Face + Username)
2. Encrypt File for Registered User
3. Authenticate & Decrypt File
4. Exit


Enter choice (1/2/3/4):  2



1. Register New User (Face + Username)
2. Encrypt File for Registered User
3. Authenticate & Decrypt File
4. Exit


Enter choice (1/2/3/4):  3


Detected username: 'sathya'
Selected file: 'sathya_hackppt.pdf.enc'

1. Register New User (Face + Username)
2. Encrypt File for Registered User
3. Authenticate & Decrypt File
4. Exit


Enter choice (1/2/3/4):  4
